In [1]:
import arcpy
import os
import time
from arcpy.ia import ExportTrainingDataForDeepLearning
from arcpy.sa import Reclassify, RemapValue

# === Your inputs ===
gdb_path = r"C:\Users\ss2596\The University of Waikato\Lars Brabyn - Skye\00 Jake Overton\Njoko_Map\Njoko_Map.gdb"
roads_fc = os.path.join(gdb_path, "roads_sample")  # polyline feature class
imagery = r"D:\Njoko_Raster_L18\3band\njoko_5-7_3band.tif"  # high-res raster
output_folder = r"C:\Users\ss2596\Documents\njoko training\Road_test6"

if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print(f"Created output folder: {output_folder}")

arcpy.env.overwriteOutput = True
arcpy.env.snapRaster = imagery
arcpy.env.extent = imagery

# Get cell size and set environment
print("Getting raster cell size...")
cell_size_result = arcpy.GetRasterProperties_management(imagery, "CELLSIZEX")
cell_size = float(cell_size_result.getOutput(0))
arcpy.env.cellSize = cell_size
print(f"Raster cell size set to: {cell_size}")

roads_buffer_fc = os.path.join(gdb_path, "roads_buffer")  # buffered polygons
roads_raster = os.path.join(gdb_path, "roads_raster")  # raster inside GDB
roads_raster_reclass_tif = os.path.join(output_folder, "roads_raster_reclass.tif")  # TIFF output

# Add "Class" field if missing and assign class = 1
print("Checking for 'Class' field and assigning class values...")
fields = [f.name for f in arcpy.ListFields(roads_fc)]
if "Class" not in fields:
    arcpy.AddField_management(roads_fc, "Class", "SHORT")
    print("Added 'Class' field.")
with arcpy.da.UpdateCursor(roads_fc, ["Class"]) as cursor:
    count = 0
    for row in cursor:
        row[0] = 1
        cursor.updateRow(row)
        count += 1
print(f"Assigned class '1' to {count} features.")

# Delete existing buffer if exists
if arcpy.Exists(roads_buffer_fc):
    arcpy.Delete_management(roads_buffer_fc)
    print(f"Deleted existing buffer feature class: {roads_buffer_fc}")

# Buffer roads by 3 meters
buffer_distance = "3 Meters"
print(f"Buffering roads by {buffer_distance}...")
arcpy.Buffer_analysis(roads_fc, roads_buffer_fc, buffer_distance)
print("Buffering complete.")

# Convert buffered polygons to raster
print("Converting buffered polygons to raster...")
arcpy.PolygonToRaster_conversion(
    in_features=roads_buffer_fc,
    value_field="Class",
    out_rasterdataset=roads_raster,
    cell_assignment="MAXIMUM_AREA",
    cellsize=cell_size
)
print(f"Buffered roads raster created in geodatabase: {roads_raster}")

# Check out Spatial Analyst extension
arcpy.CheckOutExtension("Spatial")

# Reclassify raster: 1 → 1, NoData → 0
print("Reclassifying raster...")
remap = RemapValue([[1, 1], ["NODATA", 0]])
reclass_raster = Reclassify(roads_raster, "VALUE", remap, "NODATA")

print(f"Saving reclassified raster as TIFF: {roads_raster_reclass_tif}...")
reclass_raster.save(roads_raster_reclass_tif)

print("Building raster attribute table...")
arcpy.management.BuildRasterAttributeTable(roads_raster_reclass_tif, "Overwrite")
print("Raster attribute table built.")

print("Starting export of training data...")

# Run export synchronously with progress dots
import threading

def export_training_data():
    ExportTrainingDataForDeepLearning(
        in_raster=imagery,
        in_class_data=roads_raster_reclass_tif,
        out_folder=output_folder,
        tile_size_x=512,
        tile_size_y=512,
        stride_x=256,
        stride_y=256,
        class_value_field=None,
        buffer_radius=0,
        metadata_format="CLASSIFIED_TILES",
        start_index=0,
        processing_mode="PROCESS_AS_MOSAICKED_IMAGE",
        image_chip_format="TIFF",
        output_nofeature_tiles=False  # Exclude tiles with no roads
    )

# Since threading caused issues before, let's run directly:
export_training_data()

print("Training data export complete.")


Created output folder: C:\Users\ss2596\Documents\njoko training\Road_test6
Getting raster cell size...
Raster cell size set to: 1.19432856695587
Checking for 'Class' field and assigning class values...
Assigned class '1' to 51 features.
Buffering roads by 3 Meters...
Buffering complete.
Converting buffered polygons to raster...
Buffered roads raster created in geodatabase: C:\Users\ss2596\The University of Waikato\Lars Brabyn - Skye\00 Jake Overton\Njoko_Map\Njoko_Map.gdb\roads_raster
Reclassifying raster...
Saving reclassified raster as TIFF: C:\Users\ss2596\Documents\njoko training\Road_test6\roads_raster_reclass.tif...
Building raster attribute table...
Raster attribute table built.
Starting export of training data...
Training data export complete.
